In [1]:
!pip install -Uqq rfdetr pybboxes supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import os
import cv2
import pandas as pd
import numpy as np
import wandb
import pybboxes as pbx
import pybboxes.functional as pbf
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import supervision as sv
from tqdm.auto import tqdm
from rfdetr import RFDETRBase, RFDETRLarge
from pprint import pprint

2025-05-03 18:33:22.858927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746297203.047253      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746297203.099458      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

wandb.login(key = secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abisoye to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [5]:
[i for i in range(448, 56*20, 56)]

[448, 504, 560, 616, 672, 728, 784, 840, 896, 952, 1008, 1064]

In [6]:
model = RFDETRLarge()


model.train(dataset_dir='/kaggle/input/amini-cocoa-disease-coco-dataset/amini cocoa disease coco dataset', 
            epochs=9, batch_size=2, grad_accum_steps=8,
            lr=1e-4, resolution = 448,
           wandb = True, project = 'Amini cocoa rf-detr',
           run = '9 epochs from beginning'
           )

rf-detr-large.pth: 100%|██████████| 1.46G/1.46G [00:17<00:00, 89.6MiB/s]


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loading pretrain weights


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in output/wandb/run-20250503_183358-2ldjdbyc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run 9 epochs from beginning
wandb: ⭐️ View project at https://wandb.ai/abisoye/Amini%20cocoa%20rf-detr
wandb: 🚀 View run at https://wandb.ai/abisoye/Amini%20cocoa%20rf-detr/runs/2ldjdbyc


W&B logging initialized. To monitor logs, open https://wandb.ai/abisoye/Amini%20cocoa%20rf-detr/runs/2ldjdbyc.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=3, grad_accum_steps=8, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=2, weight_decay=0.0001, epochs=9, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-large.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_base', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedforward=2048, hidden_dim=384, sa_nheads=12, ca_nheads=24, num_queries=30

fatal: not a git repository (or any of the parent directories): .git


Epoch: [0]  [  0/311]  eta: 1:20:09  lr: 0.000100  class_error: 52.31  loss: 16.7025 (16.7025)  loss_ce: 0.9687 (0.9687)  loss_bbox: 2.2348 (2.2348)  loss_giou: 0.8842 (0.8842)  loss_ce_0: 0.9298 (0.9298)  loss_bbox_0: 2.4097 (2.4097)  loss_giou_0: 0.9362 (0.9362)  loss_ce_1: 0.9694 (0.9694)  loss_bbox_1: 2.2286 (2.2286)  loss_giou_1: 0.8714 (0.8714)  loss_ce_enc: 0.9258 (0.9258)  loss_bbox_enc: 2.3868 (2.3868)  loss_giou_enc: 0.9571 (0.9571)  loss_ce_unscaled: 0.9687 (0.9687)  class_error_unscaled: 52.3077 (52.3077)  loss_bbox_unscaled: 0.4470 (0.4470)  loss_giou_unscaled: 0.4421 (0.4421)  cardinality_error_unscaled: 1748.0000 (1748.0000)  loss_ce_0_unscaled: 0.9298 (0.9298)  loss_bbox_0_unscaled: 0.4819 (0.4819)  loss_giou_0_unscaled: 0.4681 (0.4681)  cardinality_error_0_unscaled: 2089.5000 (2089.5000)  loss_ce_1_unscaled: 0.9694 (0.9694)  loss_bbox_1_unscaled: 0.4457 (0.4457)  loss_giou_1_unscaled: 0.4357 (0.4357)  cardinality_error_1_unscaled: 1334.5000 (1334.5000)  loss_ce_enc_uns

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:            Loss/Test █▄▃▂▄▂▁▃▄
wandb:           Loss/Train █▄▃▂▂▁▂▂▁
wandb:    Metrics/Base/AP50 ▁▆█▇█▆▇██
wandb: Metrics/Base/AP50_90 ▁▆▇██▆███
wandb: Metrics/Base/AR50_90 ▁▅▇▇▇▇█▇▇
wandb:     Metrics/EMA/AP50 ▁▆▇▇█████
wandb:  Metrics/EMA/AP50_90 ▁▆▇▇▇████
wandb:  Metrics/EMA/AR50_90 ▁▆▇▇▇▇██▇
wandb:                epoch ▁▂▃▄▅▅▆▇█
wandb: 
wandb: Run summary:
wandb:            Loss/Test 6.74589
wandb:           Loss/Train 6.01317
wandb:    Metrics/Base/AP50 0.78086
wandb: Metrics/Base/AP50_90 0.54817
wandb: Metrics/Base/AR50_90 0.43859
wandb:     Metrics/EMA/AP50 0.80309
wandb:  Metrics/EMA/AP50_90 0.57591
wandb:  Metrics/EMA/AR50_90 0.44415
wandb:                epoch 8
wandb: 
wandb: 🚀 View run 9 epochs from beginning at: https://wandb.ai/abisoye/Amini%20cocoa%20rf-detr/runs/2ldjdbyc
wandb: ⭐️ View project at: https://wandb.ai/abisoye/Amini%20cocoa%20rf-detr
wand